# 📚 인공지능기획및실습 RAG과제 _특허 질의 검색 시스템

인공지능기획및실습_202284045 강하나

---
### ✅ 사용 예시:
- 출원인이 '티쓰리큐 주식회사'인 특허를 알려줘
- 출원일이 2023.03.01 - 2023.03.20 안에 있는 것들 알려줘
- 출원인 국적 한국 몇명인지 알려줘
---

## 🔹 1. 필수 패키지 로드 및 엑셀 업로드

In [9]:
import pandas as pd
import re
from google.colab import files

# 엑셀 파일 업로드
uploaded = files.upload()

Saving 과제데이터.xlsx to 과제데이터 (1).xlsx


## 🔹 2. 엑셀 파일 읽기 및 키워드 매핑 설정

In [10]:
# 업로드한 파일명을 자동 인식하여 불러오기
file_path = list(uploaded.keys())[0]
df = pd.read_excel(file_path)

# 질의 분석을 위한 키워드 매핑
keyword_column_map = {
    "출원인": "출원인",
    "출원인 원문": "출원인(원문)",
    "출원인 국적": "출원인 국적",
    "발명자": "발명자",
    "출원일": "출원일",
    "출원번호": "출원번호",
    "등록번호": "등록번호",
    "공개번호": "공개번호",
    "공개일": "공개일",
    "공고번호": "공고번호",
    "메인 IPC": "메인IPC2",
    "전체 IPC": "전체 IPC",
    "요약": "요약",
    "발명의 명칭": "발명의 명칭",
    "대표청구항": "대표청구항",
    "청구항 수": "청구항 수",
    "법적상태": "법적상태",
    "현재권리자": "현재권리자",
    "문헌종류 코드": "문헌종류 코드",
    "피인용문헌 수": "피인용문헌 수",
    "국가별 피특허인용 수": "국가별 피특허인용 수"
}

# 국적 맵핑
nationality_map = {"한국": "KR", "미국": "US", "일본": "JP", "중국": "CN"}

## 🔹 3. 질문 분석 함수 정의

In [11]:
def clean_question_text(text):
    replacements = {
        # 복합 키워드 우선 처리
        "출원인 국적이 ": "출원인 국적 ",
        "출원인 국적은 ": "출원인 국적 ",
        "출원인 국적인 ": "출원인 국적 ",
        "출원인 국적을 ": "출원인 국적 ",
        "출원인 국적 한국인": "출원인 국적 한국",
        "출원인 국적 한국 몇명인지": "출원인 국적 한국 몇명",  # ✅ 추가
        "몇명인지": "몇명",
        "몇 명인지": "몇명",

        # 일반 조사 제거
        "이 ": " ", "가 ": " ", "에 ": " ", "는 ": " ", "을 ": " ", "를 ": " ",
        "특허 ": " ", "사람 ": " ",
        "보여줘": "", "알려줘": "", "찾아줘": "", "포함된": "",
        "자료": "", "목록": ""
    }
    for k, v in replacements.items():
        text = text.replace(k, v)
    return text.strip()


In [12]:
def parse_question(question):
    question = clean_question_text(question)
    print("[전처리 결과]:", question)
    ipc_match = re.search(r"[A-H][0-9]{2}[A-Z]-[0-9]{2,3}/[0-9]{2}", question)
    sorted_keywords = sorted(keyword_column_map.keys(), key=lambda x: -len(x))
    for keyword in sorted_keywords:
        if keyword in question:
            col = keyword_column_map[keyword]
            if col == "출원일" and re.search(r"\d{4}\.\d{2}\.\d{2}", question):
                return col, "DATE_RANGE"
            if ipc_match:
                return col, ipc_match.group()
            if "'" in question:
                value = question.split("'")[1]
            else:
                value = question.replace(keyword, "").strip().split()[0]
            return col, value
    return None, None

def normalize_value(col, value):
    if col == "출원인 국적":
        return nationality_map.get(value, value)
    return value

## 🔹 4. 질의 처리 및 결과 출력 함수

In [13]:
def search_and_print(df, question):
    col, value = parse_question(question)
    if col is None:
        print("⚠️ 어떤 조건을 찾을지 질문에서 이해하지 못했어요.")
        return
    value = normalize_value(col, value)

    if col == "출원일" and value == "DATE_RANGE":
        try:
            dates = re.findall(r"\d{4}\.\d{2}\.\d{2}", question)
            if len(dates) == 2:
                start_date = pd.to_datetime(dates[0])
                end_date = pd.to_datetime(dates[1])
                df[col] = pd.to_datetime(df[col], errors='coerce')
                result = df[(df[col] >= start_date) & (df[col] <= end_date)]
            else:
                print("⚠️ 날짜가 2개가 아닙니다.")
                return
        except Exception as e:
            print("⚠️ 날짜 처리 중 오류가 발생했습니다:", e)
            return
    else:
        result = df[df[col].astype(str).str.contains(value, regex=False, na=False)]

    # ✅ 수량 질의 감지
    if any(k in question for k in ["몇", "몇개", "몇 명", "개야", "얼마나"]):
        print(f"🔢 총 {len(result)}개가 검색되었습니다.")
        return

    if result.empty:
        print(f"❌ '{value}'(이)가 포함된 결과를 찾을 수 없습니다.")
    else:
        print(f"📌 질문: {question}\n📄 검색 결과:")
        for _, row in result.iterrows():
            print("─────────────────────────────")
            print("• 출원번호 :", row.get("출원번호", "정보 없음"))
            print("• 출원일   :", row.get("출원일", "정보 없음"))
            print("• 메인 IPC :", row.get("메인IPC2", "정보 없음"))
            print("• 전체 IPC :", row.get("전체 IPC", "정보 없음"))
            print("• 발명의 명칭:", row.get("발명의 명칭", "정보 없음"))
            print("• 요약정보 :", row.get("요약", "정보 없음"))
            print("• 법적상태 :", row.get("법적상태", "정보 없음"))
            print("• 출원인   :", row.get("출원인", "정보 없음"))
            print("• 발명자   :", row.get("발명자", "정보 없음"))

## 🔹 5. 질문 입력 및 실행

In [14]:
question = input("자연어 질문을 입력하세요: ")
search_and_print(df, question)

자연어 질문을 입력하세요: 출원인 국적 한국 몇명인지 알려줘
[전처리 결과]: 출원인 국적 한국 몇명
🔢 총 2370개가 검색되었습니다.
